In [9]:
import requests
import json
import csv
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time

def get_keystr(driver):
    driver.get(url)
    driver.refresh()
    modal_locator = (By.CLASS_NAME, 'bootbox')
    modal = WebDriverWait(driver, 30).until(EC.presence_of_element_located(modal_locator))
    close_button_locator = (By.CLASS_NAME, 'btn-danger.bootbox-accept')
    time.sleep(5)
    close_button = WebDriverWait(modal, 30).until(EC.element_to_be_clickable(close_button_locator))
    close_button.click()
    search_button_locator = (By.CSS_SELECTOR, 'a.leaflet-control-expand.text-dark.search')
    search_button = WebDriverWait(driver, 30).until(EC.element_to_be_clickable(search_button_locator))
    time.sleep(5)
    search_button.click()
    time.sleep(5)
    modal_locator1 = (By.CLASS_NAME, 'bootbox.modal.fade.show')
    modal1 = WebDriverWait(driver, 30).until(EC.presence_of_element_located(modal_locator1))
    close_button_locator1 = (By.CLASS_NAME, 'btn.btn-danger.bootbox-accept')
    close_button1 = WebDriverWait(modal1, 30).until(EC.element_to_be_clickable(close_button_locator1))
    close_button1.click()
    networkScript = """
    var performance = window.performance || window.webkitPerformance || {};
    var network = performance.getEntries() || {};
    return network;
    """
    networkRequests = driver.execute_script(networkScript)
    URLs = str([request['name'] for request in networkRequests if request['name'].split("/")[-1].startswith("TGL")])
    start = URLs.find('str=')+4 
    end = URLs.find('&jsonp')
    keystr = URLs[start:end]
    return keystr
cookies = {
    '_ga': 'GA1.2.2139804078.1676556517',
    '_gid': 'GA1.2.651109627.1679290300',
    '_ga_14NXYHNJ60': 'GS1.1.1679290300.4.0.1679290303.0.0.0',
    'GCLB': '"ad54252a126e9a14"',
    'ai_user': 'N8+c8Em8RvLvnVS2R58OmH|2023-03-20T05:38:00.207Z',
    'GCILB': '"592aff647ed9f325"',
}

headers = {
    'authority': 'gis.tgos.tw',
    'accept': '*/*',
    'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7',
    'cookie': '_ga=GA1.2.2139804078.1676556517; _gid=GA1.2.651109627.1679290300; _ga_14NXYHNJ60=GS1.1.1679290300.4.0.1679290303.0.0.0; GCLB="ad54252a126e9a14"; ai_user=N8+c8Em8RvLvnVS2R58OmH|2023-03-20T05:38:00.207Z; GCILB="592aff647ed9f325"',
    'referer': 'https://map.tgos.tw/',
    'sec-ch-ua': '"Google Chrome";v="111", "Not(A:Brand";v="8", "Chromium";v="111"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'script',
    'sec-fetch-mode': 'no-cors',
    'sec-fetch-site': 'same-site',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36',
}
url = 'https://map.tgos.tw/TGOSCloudMap'
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3")
chrome_options.add_argument("--incognito")
chrome_options.add_argument("--auto-open-devtools-for-tabs")
driver = webdriver.Chrome(options=chrome_options)

try:
    ################################## file path ##########################
    input_path = r'C:\Users\USER\Desktop\test.csv'
    output_path = r'C:\Users\USER\Desktop\Addresss_result.csv'
    #######################################################################
    start_time = time.time() 
    keystr = get_keystr(driver)
    print(keystr)

    df = pd.read_csv(input_path, encoding="utf-8")
    ################## column name for the address of the csv file ##################
    df1 = df['商業地址']
    #################################################################################

    arr = df1.to_numpy()

    with open(output_path, 'a', newline='', encoding="utf_8_sig") as csvfile:
        writer = csv.writer(csvfile)

        header = ['地址', 'X', 'Y']
        if csvfile.tell() == 0:
            writer.writerow(header)

        for i in arr:
            response = requests.post(
                'https://gis.tgos.tw/TGLocator/TGLocator.ashx?format=jsonp&input='+i+'&srs=EPSG:3826&ignoreGeometry=false&pnum=1&center=296884.891,2772674.703&keystr='+keystr+'&jsonp=TGOS.getJSON%5B%27sn2%27%5D',    
                cookies=cookies,
                headers=headers,
            )

            result = response.content
            result = result.decode()
            result = result.replace('TGOS.getJSON[\'sn2\'](', '')
            result = result[:-2]
            try:
                data = json.loads(result)
                if 'results' in data and data['results']:
                    result = data['results'][0]

                    if 'FULL_ADDR' in result and 'geometry' in result and 'x' in result['geometry'] and 'y' in result['geometry']:
                        adr = result['FULL_ADDR']
                        x = result['geometry']['x']
                        y = result['geometry']['y']

                        print(adr, x, y)

                else:
                    adr = "-9999999"
                    x = "-9999999"
                    y = "-9999999"

                    print(adr, x, y)

                writer.writerow([adr, x, y])

            except IndexError:
                pass

            elapsed_time = time.time() - start_time  
            if elapsed_time >= 1000:
                keystr = get_keystr(driver)
                print(keystr)
                start_time = time.time()  


except TypeError as te:
    if "can only concatenate str (not 'float') to str" in str(te):
        pass  

except Exception as e:
    print(f"An unexpected error occurred: {e}")

finally:
    driver.quit()


YqUTO2pjxdYYQgwJYzTf1B2t2ORWIhLFR%2Bs5Buvh2xI%3D
臺中市東勢區泰昌里9鄰中泰街59號 233250.13 2682979.186
-9999999 -9999999 -9999999
雲林縣褒忠鄉埔姜村10鄰太湖路1之37號 180045.75 2622690.069
臺中市大雅區秀山里8鄰平和路63號 212575.249 2680360.677
-9999999 -9999999 -9999999
高雄市苓雅區正心里23鄰安溪路6號 181344.567 2503760.893
南投縣埔里鎮牛眠里20鄰成功路22號 246169.679 2655079.589
宜蘭縣三星鄉人和村4鄰人和五路302號 314060.669 2729725.6
宜蘭縣三星鄉人和村6鄰上將路七段286號 314931.77 2730189.21
宜蘭縣三星鄉人和村1鄰月眉圳路63巷36號 315557.94 2729944.939
宜蘭縣三星鄉人和村5鄰和平路76之1號 313630.95 2729830.49
宜蘭縣三星鄉人和村5鄰和平路76之1號 313630.95 2729830.49
宜蘭縣三星鄉人和村6鄰堤防路2之1號 314992.08 2730351.049
宜蘭縣三星鄉大隱村4鄰三星路一段1號之2 323960.581 2729311.444
宜蘭縣三星鄉義德村3鄰三星路五段163號 315701.57 2728893.1
宜蘭縣三星鄉大洲村5鄰上將路二段377巷42號 323904.06 2731970.189
宜蘭縣三星鄉大洲村6鄰上將路二段399號 323806.479 2731993.779
宜蘭縣三星鄉大洲村17鄰上將路二段882號 322780.83 2731406.46
宜蘭縣三星鄉大洲村13鄰大洲路119巷30號 323639.729 2732226.689
宜蘭縣三星鄉大洲村13鄰大洲路119巷41弄5號 323587.57 2732174.069
宜蘭縣三星鄉大洲村13鄰大洲路123號 323743.12 2732253.83
宜蘭縣三星鄉大洲村5鄰大洲路57號 323940.526 2732005.83
宜蘭縣三星鄉大洲村6鄰大洲路82之12號 323906.538 2731920.646
宜蘭縣三